# Deep Learning Architectures
This notebook goes over several core deep learning models with PyTorch.

In [13]:
# Common imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torchvision.models import resnet18
import numpy as np

device = 'cpu'  # I don't have GPU

## Dense Neural Network

In [14]:
class DenseNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

## Convolutional Neural Network

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        return self.fc(x)

## Residual Neural Network

In [16]:
# Using torchvision's ResNet18 with modification for 10 classes
resnet = resnet18(pretrained=False)
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

## Autoencoder

In [17]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.encoder(x)
        x = self.decoder(x)
        return x.view(-1, 1, 28, 28)

## Diffusion Model (Simplified)

In [18]:
# Not a full diffusion model just basic denoiser
class ToyDiffusionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.denoise = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 784)
        )

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.denoise(x)
        return x.view(-1, 1, 28, 28)

## Recurrent Neural Network

In [19]:
class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=28, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        x = x.squeeze(1)
        _, h_n = self.rnn(x)
        return self.fc(h_n.squeeze(0))

## Transformer

In [20]:
class SmallTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Linear(28, 64)
        encoder_layer = TransformerEncoderLayer(d_model=64, nhead=8)
        self.encoder = TransformerEncoder(encoder_layer, num_layers=2)
        self.fc = nn.Linear(64, 10)

    def forward(self, x):
        x = x.squeeze(1)
        x = self.embed(x)
        x = self.encoder(x)
        x = x.mean(dim=1)
        return self.fc(x)

## Adversarial Learning (GAN)

In [21]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Tanh()
        )
    def forward(self, z):
        return self.model(z).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

## Reinforcement Learning (DQN-style)

In [22]:
class DQNAgent(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_dim)
        )

    def forward(self, state):
        return self.net(state)